In [1]:
from LambertSolarSystem import LambertSolarSystem
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from PyKEP import AU, DAY2SEC, DAY2YEAR, MU_SUN, epoch
from PyKEP.orbit_plots import plot_planet, plot_lambert
import PyKEP as pkp
import numpy as np


system = LambertSolarSystem(87464)

posFunc = system.getPositionsFunction()
velFunc = system.getVelocityFunction()

 # finds the optimal transfer between the planets
 Takes a start value t and two planets


In [6]:
# ----------------------- First init ---------------------------------------------------
SEC2YEAR = 1./DAY2YEAR*DAY2SEC
t = 14.32
planetsPos0 = posFunc(t)[:]
planetsVel0 = velFunc(t)[:]

A = 0; B = 4; C = 5
planets = np.array((A,B,C))
# Add 3. dimension
pos0 = np.r_[planetsPos0, [np.zeros(7)]]
vel0 = np.r_[planetsVel0, [np.zeros(7)]]
rA = pos0[:,A] * AU
vA = vel0[:,A] * AU / SEC2YEAR
rB = pos0[:,B] * AU
vB = vel0[:,B] * AU / SEC2YEAR
rC = pos0[:,C] * AU
vC = vel0[:,C] * AU / SEC2YEAR

mu_star = system.starMass * MU_SUN
mu_planet = system.mass[planets]  * MU_SUN
radius = system.radius[planets] * 1000
safe_radius = radius *1.2

planetA = pkp.planet.keplerian(epoch(0), rA, vA, mu_star, mu_planet[0], radius[0], safe_radius[0], 'A')
planetB = pkp.planet.keplerian(epoch(0), rB, vB, mu_star, mu_planet[1], radius[1], safe_radius[1], 'B')
planetC = pkp.planet.keplerian(epoch(0), rC, vC, mu_star, mu_planet[2], radius[2], safe_radius[2], 'C')

res = 10000
time_array = np.linspace(0.2,4,10000)
time_use_arr = np.zeros_like(time_array)
v_arr1 = np.zeros_like(time_array)
vBin_arr = np.zeros((res,3))
# ----------------------Find best GA-times ----------------------

for j,time_use in enumerate(time_array):
    t1 = epoch(0)            #relative to t0
    t2 = epoch(time_use/DAY2YEAR)   #relative to t0
    dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC


    r1, v1 = planetA.eph(t1)
    r2, v2 = planetB.eph(t2)
    
    l = pkp.lambert_problem(r1,r2,dt,mu_star)

    #vA = l.get_v1()
    vB = np.array(l.get_v2())
    vel =  np.linalg.norm(vA) 
    time_use_arr[j] = time_use
    v_arr1[j] =  vel
    vBin_arr[j] = vB


best_v = np.amin(v_arr1)
print 'min v' ,best_v

i = np.argmin(v_arr1)
vBin = vBin_arr[i]

optimal_time1 = time_use_arr[i]
print 'optimal time use:', optimal_time1


time_array = np.linspace(2,8,res)
time_use_arr = np.zeros_like(time_array)
v_arr2 = np.zeros_like(time_array)
vBout_arr = np.zeros((res,3))

for j,time_use in enumerate(time_array):
    t1 = epoch(optimal_time1)            #relative to t0
    t2 = epoch((optimal_time1+time_use)/DAY2YEAR)   #relative to t0
    dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC


    r1, v1 = planetB.eph(t1)
    r2, v2 = planetC.eph(t2)
    
    l = pkp.lambert_problem(r1,r2,dt,mu_star)
    
    vB = np.array(l.get_v1())[0]
    #vC = l.get_v2()
    vel =  np.linalg.norm(vB-vBin)
    #print vB

    vBout_arr[j] = vB
    
    time_use_arr[j] = time_use
    v_arr2[j] =  vel

best_v = np.amin(v_arr2)
print 'min v' ,best_v

i = np.argmin(v_arr2)

vBout = vBout_arr[i]
print vBin, vBout
optimal_time2 = time_use_arr[i]
print 'optimal time use:', optimal_time2


min v 22277.3503102
optimal time use: 0.2
min v 72795.9299732
[ 84550.0169419    3542.72478661      0.        ] [ 13564.44184106 -12590.95266982      0.        ]
optimal time use: 8.0


In [7]:
fig = plt.figure()
ax = fig.gca(projection = '3d')

t1 = epoch(0)            #relative to t0
t2 = epoch(optimal_time1/DAY2YEAR)   #relative to t0
dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC

r1, v1 = planetA.eph(t1)
r2, v2 = planetB.eph(t2)
l = pkp.lambert_problem(r1,r2,dt,mu_star)
norm = np.linalg.norm
print norm(l.get_v1())
print norm(v1)

plot_planet(planetA, t0 = epoch(0), legend = True, units = AU, ax = ax)
plot_planet(planetB, t0 = epoch(optimal_time1/DAY2YEAR), legend = True, units = AU, ax = ax)

for sol in range(l.get_Nmax()+1):
    plot_lambert(l, sol = sol, legend=True, units =AU, ax=ax)

86901.1321421
22277.3503102


In [8]:
t1 = epoch(optimal_time1/DAY2YEAR)            #relative to t0
t2 = epoch((optimal_time1+optimal_time2)/DAY2YEAR)   #relative to t0
dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC

r1, v1 = planetB.eph(t1)
r2, v2 = planetC.eph(t2)
l = pkp.lambert_problem(r1,r2,dt,mu_star)
norm = np.linalg.norm
print norm(l.get_v1())
print norm(v1)

plot_planet(planetB, t0 = t1, legend = True, units = AU, ax = ax)
plot_planet(planetC, t0 = t2, legend = True, units = AU, ax = ax)

for sol in range(l.get_Nmax()+1):
    plot_lambert(l, sol = sol, legend=True, units =AU, ax=ax)

plt.show()

26370.8520576
17409.220093


#### Have found a way to get best lambert from one planet to another. Now lets find the best from one to another via a third!
